In [1]:
import pandas as pd
from tqdm import tqdm

## Load Alarms with Lavori Programmati

In [2]:
mob = pd.read_parquet("alarms datasets/mob/[new]20230101-20240101_inpas_mob_preprocess__an__last_event__last_event__ext1.parquet")

In [5]:
adsl = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__adsl__last_event__last_event__ext1.parquet")
pdh = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__pdh__last_event__last_event__ext1.parquet")
ptn = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__ptn__last_event__last_event__ext1.parquet")
sdh = pd.read_parquet("alarms datasets/tx/[new]20230101-20240101_inpas_tx_preprocess__sdh__last_event__last_event__ext1.parquet")

In [3]:
mob["link_type"] = "None"

In [4]:
mob = mob[["alarm_id", "std_probable_cause_no","link_type", "lp_id", "lp_type"]]

In [6]:

adsl = adsl[["alarm_id", "std_probable_cause_no","link_type", "lp_id", "lp_type"]]
pdh = pdh[["alarm_id", "std_probable_cause_no", "link_type","lp_id", "lp_type"]]
ptn = ptn[["alarm_id", "std_probable_cause_no","link_type", "lp_id", "lp_type"]]
sdh = sdh[["alarm_id", "std_probable_cause_no", "link_type","lp_id", "lp_type"]]

In [7]:
alarms_slogan_lavori_programmati = pd.concat([mob, adsl, pdh, ptn, sdh])

In [8]:
alarms_slogan_lavori_programmati["link_type"] = alarms_slogan_lavori_programmati["link_type"].fillna("None")

In [9]:
alarms_slogan_lavori_programmati.to_parquet("alarms datasets/alarms_slogan_lavori_programmati.parquet")

## Merge Clusters and Alarms with Lavori Programmati

In [10]:
clusters = pd.read_parquet("real-time clusters/20230101-20240101_real_time_clusters_filtered.parquet")

In [11]:
alarms_slogan_lavori_programmati = pd.read_parquet("alarms datasets/alarms_slogan_lavori_programmati.parquet")

In [12]:
clusters_with_lavori_programmati = pd.merge(clusters, alarms_slogan_lavori_programmati, on="alarm_id", how="left")

In [13]:
clusters_with_lavori_programmati = clusters_with_lavori_programmati[
    ["alarm_id", "cluster_id2", "std_probable_cause_no","link_type", "ne_type", "lp_type"]
]

In [14]:
clusters_with_lavori_programmati

,alarm_id,cluster_id2,std_probable_cause_no,link_type,ne_type,lp_type
0,COS1P;136885076,202301010000_17,apparato isolato,None,ad16,None
1,COS1P;136885115,202301010000_17,apparato isolato,None,ad16,None
2,COS1P;136884642,202301010000_26,apparato isolato,None,adm4,None
3,COS1P;136884882,202301010000_26,apparato isolato,None,ad16,None
4,COS1P;136884883,202301010000_26,apparato isolato,None,ad16,None
...,...,...,...,...,...,...
11933090,EFMTX;6591f14afeea70000175b0cd,202312312346_17,fuori servizio,vc1,None,None
11933091,EFMTX;6591f14aad7bae0001861f6b,202312312346_17,fuori servizio,vc1,None,None
11933092,EFMTX;6591f14ad8beb800016debf6,202312312346_17,fuori servizio,None,None,None
11933093,EFMTX;6591ef8b975f5100015a479b,202312312342_10,fuori servizio,vc1,None,None


In [15]:
clusters_with_lavori_programmati['std_probable_cause_no'] = clusters_with_lavori_programmati['std_probable_cause_no'].fillna('None')

In [16]:
clusters_with_lavori_programmati

,alarm_id,cluster_id2,std_probable_cause_no,link_type,ne_type,lp_type
0,COS1P;136885076,202301010000_17,apparato isolato,None,ad16,None
1,COS1P;136885115,202301010000_17,apparato isolato,None,ad16,None
2,COS1P;136884642,202301010000_26,apparato isolato,None,adm4,None
3,COS1P;136884882,202301010000_26,apparato isolato,None,ad16,None
4,COS1P;136884883,202301010000_26,apparato isolato,None,ad16,None
...,...,...,...,...,...,...
11933090,EFMTX;6591f14afeea70000175b0cd,202312312346_17,fuori servizio,vc1,None,None
11933091,EFMTX;6591f14aad7bae0001861f6b,202312312346_17,fuori servizio,vc1,None,None
11933092,EFMTX;6591f14ad8beb800016debf6,202312312346_17,fuori servizio,None,None,None
11933093,EFMTX;6591ef8b975f5100015a479b,202312312342_10,fuori servizio,vc1,None,None


In [17]:
filtered_df = clusters_with_lavori_programmati[clusters_with_lavori_programmati['lp_type'].apply(lambda x: x != 'None')]
filtered_df

,alarm_id,cluster_id2,std_probable_cause_no,link_type,ne_type,lp_type
0,COS1P;136885076,202301010000_17,apparato isolato,None,ad16,None
1,COS1P;136885115,202301010000_17,apparato isolato,None,ad16,None
2,COS1P;136884642,202301010000_26,apparato isolato,None,adm4,None
3,COS1P;136884882,202301010000_26,apparato isolato,None,ad16,None
4,COS1P;136884883,202301010000_26,apparato isolato,None,ad16,None
...,...,...,...,...,...,...
11933090,EFMTX;6591f14afeea70000175b0cd,202312312346_17,fuori servizio,vc1,None,None
11933091,EFMTX;6591f14aad7bae0001861f6b,202312312346_17,fuori servizio,vc1,None,None
11933092,EFMTX;6591f14ad8beb800016debf6,202312312346_17,fuori servizio,None,None,None
11933093,EFMTX;6591ef8b975f5100015a479b,202312312342_10,fuori servizio,vc1,None,None


In [18]:
clusters_with_lavori_programmati['lp_type'] = clusters_with_lavori_programmati['lp_type'].replace([None], ['None'])

In [19]:
clusters_with_lavori_programmati['lp_type'] = clusters_with_lavori_programmati['lp_type'].str.replace(r"\[|\]|\'", "", regex=True)

In [20]:
unique_lp_type_values = clusters_with_lavori_programmati['lp_type'].apply(lambda x: str(x)).unique()
unique_lp_type_values

array(['None', 'APPARATI', 'CAVO', 'REIX/UPGRADE', 'DWDM/PTN',
       'CAVO APPARATI', 'CAMBI RELEASE', 'CAVO DWDM/PTN', 'VARIE'],
      dtype=object)

In [21]:
clusters_with_lavori_programmati

,alarm_id,cluster_id2,std_probable_cause_no,link_type,ne_type,lp_type
0,COS1P;136885076,202301010000_17,apparato isolato,None,ad16,None
1,COS1P;136885115,202301010000_17,apparato isolato,None,ad16,None
2,COS1P;136884642,202301010000_26,apparato isolato,None,adm4,None
3,COS1P;136884882,202301010000_26,apparato isolato,None,ad16,None
4,COS1P;136884883,202301010000_26,apparato isolato,None,ad16,None
...,...,...,...,...,...,...
11933090,EFMTX;6591f14afeea70000175b0cd,202312312346_17,fuori servizio,vc1,None,None
11933091,EFMTX;6591f14aad7bae0001861f6b,202312312346_17,fuori servizio,vc1,None,None
11933092,EFMTX;6591f14ad8beb800016debf6,202312312346_17,fuori servizio,None,None,None
11933093,EFMTX;6591ef8b975f5100015a479b,202312312342_10,fuori servizio,vc1,None,None


In [22]:
clusters_with_lavori_programmati['ne_type_slogan_link_type'] = clusters_with_lavori_programmati.apply(lambda row: str(row.ne_type) + '_' + str(row.std_probable_cause_no + '_' + str(row.link_type)), axis=1)

In [23]:
clusters_with_lavori_programmati

,alarm_id,cluster_id2,std_probable_cause_no,link_type,ne_type,lp_type,ne_type_slogan_link_type
0,COS1P;136885076,202301010000_17,apparato isolato,None,ad16,None,ad16_apparato isolato_None
1,COS1P;136885115,202301010000_17,apparato isolato,None,ad16,None,ad16_apparato isolato_None
2,COS1P;136884642,202301010000_26,apparato isolato,None,adm4,None,adm4_apparato isolato_None
3,COS1P;136884882,202301010000_26,apparato isolato,None,ad16,None,ad16_apparato isolato_None
4,COS1P;136884883,202301010000_26,apparato isolato,None,ad16,None,ad16_apparato isolato_None
...,...,...,...,...,...,...,...
11933090,EFMTX;6591f14afeea70000175b0cd,202312312346_17,fuori servizio,vc1,None,None,None_fuori servizio_vc1
11933091,EFMTX;6591f14aad7bae0001861f6b,202312312346_17,fuori servizio,vc1,None,None,None_fuori servizio_vc1
11933092,EFMTX;6591f14ad8beb800016debf6,202312312346_17,fuori servizio,None,None,None,None_fuori servizio_None
11933093,EFMTX;6591ef8b975f5100015a479b,202312312342_10,fuori servizio,vc1,None,None,None_fuori servizio_vc1


## Create a Dataframe with Clusters and "GUASTO CAVO" label

In [24]:
clusters_grouped = clusters_with_lavori_programmati.groupby(["cluster_id2"])

### Labeling Rule

- GUASTO CAVO = 1 : if at least one alarm inside the cluster has a lp_type == 'CAVO'
- GUASTO CAVO = 0 : if there are no alarms with a lp_type == 'CAVO'
- No labeling (cluster not in the dataframe) : if whole number of alarms in the cluster have a lp_type == 'None' (no lavoro programmato info)

In [25]:
def check_cavo_alarm(group):
    if all(x == 'None' for x in group['lp_type'].values):
        return None
    return any(sublist == 'CAVO' for sublist in group['lp_type'].values if sublist is not None)

temp_df = clusters_grouped.apply(lambda group: pd.Series({
    'cluster_id2': group.name,
    'GUASTO CAVO': check_cavo_alarm(group)
}))

new_df = temp_df.dropna().reset_index(drop=True)

C:\Users\Nicolò Caradonna\AppData\Local\Temp\ipykernel_5284\1928354519.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  temp_df = clusters_grouped.apply(lambda group: pd.Series({


In [26]:
new_df.to_csv("real-time clusters/20230101-20240101_real_time_clusters_filtered_guasto_cavo.csv", index=False)

## Create Cluster/Slogan Bitmap for lavoro programmato clusters

In [27]:
new_df = pd.read_csv("real-time clusters/20230101-20240101_real_time_clusters_filtered_guasto_cavo.csv")

In [28]:
new_df

,cluster_id2,GUASTO CAVO
0,202301090504_2,False
1,202301090504_3,False
2,202301090506_10,False
3,202301090506_15,False
4,202301090506_16,False
...,...,...
6668,202312210210_1,True
6669,202312210322_7,True
6670,202312210434_5,True
6671,202312210444_3,True


In [29]:
filtered_clusters = clusters_with_lavori_programmati[clusters_with_lavori_programmati['cluster_id2'].isin(new_df['cluster_id2'])]

In [30]:
filtered_clusters

,alarm_id,cluster_id2,std_probable_cause_no,link_type,ne_type,lp_type,ne_type_slogan_link_type
153504,COS1P;137150277,202301090504_2,loss of signal los,rt00,rxm3,None,rxm3_loss of signal los_rt00
153505,COS1P;137150264,202301090504_3,scheda guasta,None,pxm3,None,pxm3_scheda guasta_None
153506,COS1P;137150265,202301090504_3,scheda guasta,None,pxm3,None,pxm3_scheda guasta_None
153518,COS1P;137150310,202301090504_2,fuori servizio,odu0,None,APPARATI,None_fuori servizio_odu0
153519,COS1P;137150415,202301090504_2,loss of signal los,och,None,APPARATI,None_loss of signal los_och
...,...,...,...,...,...,...,...
11701885,EFMTX;6583bd3e171e780001883061,202312210518_6,fuori servizio,vc1,None,None,None_fuori servizio_vc1
11701900,EFMTX;6583bd3d469e8c0001370d40,202312210518_6,fuori servizio,vc3,None,None,None_fuori servizio_vc3
11701901,EFMTX;6583bd3ec3980b0001211344,202312210518_6,fuori servizio,vc1,None,None,None_fuori servizio_vc1
11719565,EFMTX;6583901bf25c6e000142fd31,202312210200_3,fuori servizio,odu0,None,None,None_fuori servizio_odu0


In [31]:
grouped_clusters = filtered_clusters.groupby("cluster_id2")
unique_slogans = clusters_with_lavori_programmati['ne_type_slogan_link_type'].unique()

In [32]:
len(unique_slogans)

1917

In [33]:
# Definisci una funzione per calcolare la presenza di ciascuna tripla ne_type_slogan_link_type all'interno di un gruppo
def calculate_presence(group):
    presence = {slogan: 0 for slogan in unique_slogans}
    for slogan in group['ne_type_slogan_link_type']:
        presence[slogan] = 1
    return pd.Series(presence)

tqdm.pandas()

# Applica la funzione a ciascun gruppo e crea un nuovo dataframe con i risultati
result_df = grouped_clusters.progress_apply(calculate_presence)

100%|██████████| 6673/6673 [00:07<00:00, 878.68it/s] 


In [34]:
result_df.to_parquet("result_df_gt_2_lavoriprogrammati_1917.parquet")

In [35]:
result_df

,ad16_apparato isolato_None,adm4_apparato isolato_None,lhbts_fermo parziale sito tec_None,exc_oproutgoing channel optical power out of range xexc_None,hwbts_fermo parziale gsm_None,hwbts_fermo totale gsm_None,None_fuori servizio_vc1,ad16_None_nan,ad64_None_nan,adm1_apparato isolato_None,...,None_loss of frame lof_vc3,rxm5_loss of signal los_gb100,None_problema laser tx_None,None_fuori servizio_gb100,pxm3_loss of frame lof_odu2,wdm_sef sef_r12,ptn2_signal degraded_None,oadm1_apparato isolato_None,rmx5_loss of signal los_None,adm1_sef intermittente sef_pre
cluster_id2,,,,,,,,,,,,,,,,,,,,,
202301090504_2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202301090504_3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202301090506_10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202301090506_15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202301090506_16,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202312210210_1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202312210322_7,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202312210434_5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
guasto_cavo = pd.read_csv("real-time clusters/20230101-20240101_real_time_clusters_filtered_guasto_cavo.csv")

In [4]:
guasto_cavo

,cluster_id2,GUASTO CAVO
0,202301090504_2,False
1,202301090504_3,False
2,202301090506_10,False
3,202301090506_15,False
4,202301090506_16,False
...,...,...
6668,202312210210_1,True
6669,202312210322_7,True
6670,202312210434_5,True
6671,202312210444_3,True


In [6]:
true_count = (guasto_cavo['GUASTO CAVO'] == True).sum()
false_count = (guasto_cavo['GUASTO CAVO'] == False).sum()
print("True count: ", true_count*100/(true_count+false_count))
print("False count: ", false_count*100/(true_count+false_count))

True count:  80.99805185074179
False count:  19.001948149258205
